In [30]:
import nltk
import numpy as np 
import pandas as pd
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
comentarios = pd.read_csv('C:/Users/Eduardo/Desktop/TCC-programa/train_gr/train.csv') #carregando dados de treino

In [32]:
print(comentarios.shape) #exibir na tela o tamanho da base de dados
comentarios.head()

(17494, 5)


,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1


In [33]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17494 entries, 0 to 17493
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review_id        17494 non-null  int64  
 1   title            17494 non-null  object 
 2   year             17316 non-null  float64
 3   user_review      17494 non-null  object 
 4   user_suggestion  17494 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 683.5+ KB


In [34]:
comentarios.user_suggestion.value_counts() #exibir números de reviews positivos e negativos

1    9968
0    7526
Name: user_suggestion, dtype: int64

In [35]:
comentarios["n_palavras"] = comentarios["user_review"].apply(lambda s: len(s.split()))
comentarios.head()

,review_id,title,year,user_review,user_suggestion,n_palavras
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1,132
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1,44
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1,70
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1,47
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1,67


## Mineracao dos dados

### Remover comentarios repitidos

In [36]:
comentarios.drop_duplicates(['user_review'], inplace=True)

In [37]:
comentarios.user_review.count()

17490

### Remover spam

In [38]:
reviewData = comentarios[["title", "user_review", "user_suggestion", "n_palavras"]].values.tolist()

In [39]:
def isEnglish(texto):
    totalChars = len(texto)
    nAsciiChars = 0
    for c in list(texto):
        if ord(c) >= 0 and ord(c) <= 128:
            nAsciiChars += 1
    return (nAsciiChars / totalChars) >= 0.5
    

In [40]:
englishReviews = []
nonEnglishReviews = []

for review in reviewData :
    if isEnglish(review[1]):
        englishReviews.append(review)
    else:
        nonEnglishReviews.append(review)
        
comentarios = pd.DataFrame(englishReviews, columns = ['titulo','user_review','user_suggestion','n_palavras'])

In [41]:
dados_bruto = pd.DataFrame(nonEnglishReviews, columns = ['titulo','user_review','user_suggestion','n_palavras'])
dados_bruto.tail()
#print(comentarios.shape)
#comentarios.head()

,titulo,user_review,user_suggestion,n_palavras
86,Bloons TD Battles,░░░░░░░░░░░█▀▀░░█░░░░░░░░░░░░▄▀▀▀▀░░░░░█▄▄░░░░...,0,12
87,Cuisine Royale,Early Access ReviewКто-то может запрыгнуть мак...,0,32
88,Cuisine Royale,"Early Access Reviewat first, i was like this▒▒...",1,47
89,School of Dragons,Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â ...,1,91
90,School of Dragons,игра хорошая играть класно и драконы прикольны...,1,80


### Padronizar

In [42]:
def padronizar(comentario):
    return comentario.lower()

In [43]:
comentarios.user_review = comentarios.user_review.apply(padronizar)

In [44]:
comentarios.head()

,titulo,user_review,user_suggestion,n_palavras
0,Spooky's Jump Scare Mansion,i'm scared and hearing creepy voices. so i'll...,1,132
1,Spooky's Jump Scare Mansion,"best game, more better than sam pepper's youtu...",1,44
2,Spooky's Jump Scare Mansion,"a littly iffy on the controls, but once you kn...",1,70
3,Spooky's Jump Scare Mansion,"great game, fun and colorful and all that.a si...",1,47
4,Spooky's Jump Scare Mansion,not many games have the cute tag right next to...,1,67


### Limpar Dados

In [45]:
import string
string.punctuation
caracteres = string.punctuation

def remover_caractere(comentario):
    sem_caractere=[palavra for palavra in comentario if palavra not in caracteres]
    words_wo_punct=''.join(sem_caractere)
    return words_wo_punct

In [46]:
comentarios.user_review = comentarios.user_review.apply(remover_caractere)
comentarios.head()

,titulo,user_review,user_suggestion,n_palavras
0,Spooky's Jump Scare Mansion,im scared and hearing creepy voices so ill pa...,1,132
1,Spooky's Jump Scare Mansion,best game more better than sam peppers youtube...,1,44
2,Spooky's Jump Scare Mansion,a littly iffy on the controls but once you kno...,1,70
3,Spooky's Jump Scare Mansion,great game fun and colorful and all thata side...,1,47
4,Spooky's Jump Scare Mansion,not many games have the cute tag right next to...,1,67


### Remover Stop Words

In [47]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Eduardo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
def remove_stopWord(comentario): 
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in comentario.split() if not i in stopwords]
    return (" ".join(palavras))

In [49]:
comentarios.user_review = comentarios.user_review.apply(remove_stopWord)
comentarios.head()

,titulo,user_review,user_suggestion,n_palavras
0,Spooky's Jump Scare Mansion,im scared hearing creepy voices ill pause mome...,1,132
1,Spooky's Jump Scare Mansion,best game better sam peppers youtube account 1...,1,44
2,Spooky's Jump Scare Mansion,littly iffy controls know play easy master ive...,1,70
3,Spooky's Jump Scare Mansion,great game fun colorful thata side note though...,1,47
4,Spooky's Jump Scare Mansion,many games cute tag right next horror tag stea...,1,67


### Lemenizar

In [50]:
def stemming(instancia):
    stemmer = SnowballStemmer('english')
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [51]:
comentarios.user_review = comentarios.user_review.apply(stemming)
comentarios.head()

,titulo,user_review,user_suggestion,n_palavras
0,Spooky's Jump Scare Mansion,im scare hear creepi voic ill paus moment writ...,1,132
1,Spooky's Jump Scare Mansion,best game better sam pepper youtub account 101...,1,44
2,Spooky's Jump Scare Mansion,litt iffi control know play easi master ive ma...,1,70
3,Spooky's Jump Scare Mansion,great game fun color thata side note though ge...,1,47
4,Spooky's Jump Scare Mansion,mani game cute tag right next horror tag steam...,1,67


## Criando modelo

**Instancia o objeto que faz a vetorização dos dados de texto**

In [52]:
vectorizer = CountVectorizer(analyzer="word", tokenizer=word_tokenize, max_features=10000)

**Aplica o vetorizador nos dados de texto**

In [53]:
freq_palavra = vectorizer.fit_transform(comentarios.user_review)
freq_palavra.shape

(17399, 10000)

**Separar dados de treino e de teste**

In [54]:
y = np.array(comentarios.user_suggestion.values)
treinox, testex, treinoy, testey = train_test_split(freq_palavra,y,test_size=0.2, random_state = 40)

**Treinar modelo**

In [55]:
treinox

<13919x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 725966 stored elements in Compressed Sparse Row format>

In [56]:
testex

<3480x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 178655 stored elements in Compressed Sparse Row format>

In [57]:
modelo = MultinomialNB()
modelo.fit(treinox,treinoy)

MultinomialNB()

In [58]:
ypm = modelo.predict(testex)
print ("Accuracy =", accuracy_score(testey,ypm))

Accuracy = 0.835919540229885


### Aplicar modelo

In [59]:
feed_back = pd.read_csv('C:/Users/Eduardo/Desktop/TCC-programa/test_gr/test.csv') #carregando dados de test
feed_back.head()

,review_id,title,year,user_review
0,1603,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B..."
1,1604,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...
2,1605,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...
3,1606,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...
4,1607,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...


In [60]:
feed_back["n_palavras"] = feed_back["user_review"].apply(lambda s: len(s.split()))
feed_back.head()

,review_id,title,year,user_review,n_palavras
0,1603,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B...",78
1,1604,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...,170
2,1605,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...,484
3,1606,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...,72
4,1607,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...,970


In [61]:
print(feed_back.shape)

(8045, 5)


In [62]:
data = feed_back[["title", "year","user_review", "n_palavras"]].values.tolist()

english = []
nonEnglish = []

for comentar in data :
    if isEnglish(comentar[2]):
        english.append(comentar)
    else:
        nonEnglish.append(comentar)
        
feed_back = pd.DataFrame(english, columns = ["title", "year","user_review", "n_palavras"])

print("COMENTARIOS REMOVIDOS: ")
nonEnglish

COMENTARIOS REMOVIDOS: 


[['Counter-Strike: Global Offensive',
  2014.0,
  "'CS:GO' <= Энэ ч үнэхээр хачин тоглоом шүү ...Тэгэхээр ийм байна. Тоглоомын эхэнд чи алан хядагчид эсвэл тусгайгийн цагдаа нар гэсэн хоёр багийн аль нэгэнд нь автоматаар хуваарилагдах болно. Хэрэв чамайг аз дайрвал, багт чинь 1 ширхэг орос тааралдана. Харин эсрэг тохиолдолд, 2-3 оростой нэг багт учирна гэж мэд! За тэгээд ... террористуудын гол даалгавар нь тун энгийн : тэсрэх бөмбөгөө замдаа гээлгүйхэн шиг авч явсаар 'Шороо Хоёр' гэдэг газарт модон texture-тай чулуун хайрцагнуудыг, харин 'Там' гэдэг газарт усан оргилуур доторх загаснуудыг дэлбэлж алах ёстой байгаа юм. Харин цагдаа нарын гол зорилго нь цаг дуусахаас өмнө очиж тэр бөмбөгийг нь салгаад хайрцагнуудыг, бас загаснуудыг аврахад оршино. Түүнчлэн, 'Албан Газар', 'Довтолгоо' зэрэг газруудад арай өөр дүрэм үйлчилдэг. Ердөө л шар өнгийн унтлагын хувцастай залуусыг чи А эсвэл B цэгүүдээс эсрэг багийнхандаа харагдалгүй нууцаар үүрч явсаар өөрсдийнхөө төрдөг дээрээ авчраад, шар зураа

In [63]:
print(feed_back.shape)
feed_back.head()

(8008, 4)


,title,year,user_review,n_palavras
0,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B...",78
1,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...,170
2,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...,484
3,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...,72
4,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...,970


In [64]:
feed_back["classificacao"] = feed_back.user_review
feed_back.head()

,title,year,user_review,n_palavras,classificacao
0,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B...",78,"Nice graphics, new maps, weapons and models. B..."
1,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...,170,I would not recommend getting into this at its...
2,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...,484,Edit 11/12/18I have tried playing CS:GO recent...
3,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...,72,The game is great. But the community is the wo...
4,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...,970,I thank TrulyRazor for buying this for me a lo...


In [65]:
feed_back.classificacao = feed_back.classificacao.apply(padronizar)
feed_back.classificacao = feed_back.classificacao.apply(remover_caractere)
feed_back.classificacao = feed_back.classificacao.apply(remove_stopWord)
feed_back.classificacao = feed_back.classificacao.apply(stemming)

In [66]:
lista = feed_back['classificacao'].tolist()
lista

['nice graphic new map weapon model develop listen custom bit develop focus much thing import focus chang tick rate match make server 128 improv vac lot two custom realli want focus sticker ui hud chang skin stop mess around weapon',
 'would recommend get current state csgo hit rock bottom valv miss promis game chang updat lack effort combat terribl cheat problem bug game year still havent fix cheat control past year play cheat gotten wors wors point end play cheater almost everi game valv implement new trust matchmak system combat issu lacklust algorithm still somehow think put year gameplay 3000 hour peopl play brand new account make sens realli dont know still put effort play game anymor usual end regret guess wish could somehow fun use play game seem day long gone',
 'edit 111218i tri play csgo recent dramat gotten wors player open obvious cheat even say chat wall aim peopl clean spray wall even dm casual far beyond use tonow know mani cheater game begin sinc csgo f2p playground ha

In [67]:
# Transforma os dados de teste em vetores de palavras.
freq_testes = vectorizer.transform(lista)

In [68]:
classe = []
for t, c in zip (lista,modelo.predict(freq_testes)):
    classe.append(np.int16(c).item())
    

In [69]:
feed_back["classificacao"] = classe
feed_back

,title,year,user_review,n_palavras,classificacao
0,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B...",78,0
1,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...,170,0
2,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...,484,0
3,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...,72,0
4,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...,970,1
...,...,...,...,...,...
8003,GUNS UP!,2017.0,Early Access ReviewGuns UP!Positive: Good Idea...,86,1
8004,GUNS UP!,2018.0,"After 170 hrs, my review. DO NOT SPEND ANY MON...",208,0
8005,GUNS UP!,2018.0,Pros:-Fun GameplayCons: -Micro %$#*ing transac...,60,0
8006,GUNS UP!,2018.0,"Actualy saucy, I definetly suggest to players ...",135,1


In [70]:
avaliacao_negativa = feed_back.query("classificacao==0")

avaliacao_negativa.to_excel('avaliacaonegativa.xlsx', sheet_name="Comentarios_negativo",header="True", index="False")


C:\Users\Eduardo\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:941: UserWarning: Ignoring URL 'http://www.youtube.com/watch?v=9I4qWvqAt78%20(youtube%20video%20of%20game%20play/bug)Game%20is%20still%20buggy%20and%20unstable.%20Notably%20(atleast%20under%20windows%2010)%20you%20litterally%20have%20to%20'fight'%20with%20origins%20for%20focus%20of%20the%20window%20if%20you%20swap%20out%20to%20anything..%20(audio%20in%20the%20background..%20want%20to%20change%20it?%20good%20luck)%202%20Headed%20Giant%20matches%20are%20back!%20w00t!%20problem.%20NO%20INGAME%20CHAT.....Actually%20there's%20no%20lobby%20of%20any%20kind%20what%20so%20ever.%20You%20can%20invite%20friends%20(from%20steam)%20into%20a%20game%20(haven't%20tried%20as%20I%20have%20no%20mtg%20friends%20on%20steam)%20and%20can't%20make%20any..%20because%20you%20cannot%20chat%20or%20interact%20with%20any%20other%20players%20aside%20from%20playing%20cards.%20Played%20opponents%20do%20not%20even%20appear%20in%20your%20steam%20history

In [71]:
avaliacao_positiva=feed_back.query("classificacao==1")

avaliacao_positiva.to_excel('avaliacaopositiva.xlsx', sheet_name="Comentarios_Positivos",header="True", index="False")
  
